In [ ]:
import astropy.coordinates as coord
import astropy.units as u
from astropy.table import Table, join, vstack
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.io import ascii

from scipy.interpolate import interp1d
from scipy.stats import binned_statistic

import gala.coordinates as gc
import gala.dynamics as gd
from gala.units import galactic
from pyia import GaiaData

In [ ]:
pm_poly = np.load('../../gd1-dr2/output/pm_poly.npy')
phi_poly = np.load('../../gd1-dr2/output/phi12_poly.npy')

In [ ]:
w = np.load('../../gd1-dr2/data/stream_model.npy')
stream_w = gd.PhaseSpacePosition(pos=w[:, :3].T*u.kpc,
                                 vel=w[:, 3:].T*u.km/u.s)
model_c = stream_w.to_coord_frame(gc.GD1)
model_c = model_c[(model_c.phi1.wrap_at(180*u.deg) < 20*u.deg) & 
                  (model_c.phi1.wrap_at(180*u.deg) > -100*u.deg)]
model_c_ref = gc.reflex_correct(model_c)

In [ ]:
stat = binned_statistic(model_c.phi1.wrap_at(180*u.deg).value,
                        model_c.radial_velocity.to(u.km/u.s).value,
                        bins=np.linspace(-140, 50, 128))
phi1_to_rv = interp1d(0.5 * (stat.bin_edges[:-1]+stat.bin_edges[1:]), 
                      stat.statistic, bounds_error=False)

In [ ]:
gd1_g = GaiaData('../../gd1-dr2/data/gd1-master.fits')

In [ ]:
gd1_c = gd1_g[gd1_g.gi_cmd_mask & gd1_g.pm_mask].get_skycoord(distance=False)
gd1_c = gd1_c.transform_to(gc.GD1)

gd1_phi1 = gd1_c.phi1.wrap_at(180*u.deg)
gd1_phi2 = gd1_c.phi2
gd1_pm1 = gd1_c.pm_phi1_cosphi2.to(u.mas/u.yr)
gd1_pm2 = gd1_c.pm_phi2.to(u.mas/u.yr)

In [ ]:
kop_vr = ascii.read("""phi1 phi2 vr err
-45.23 -0.04 28.8 6.9
-43.17 -0.09 29.3 10.2
-39.54 -0.07 2.9  8.7
-39.25 -0.22 -5.2 6.5
-37.95 0.00 1.1   5.6
-37.96 -0.00 -11.7 11.2
-35.49 -0.05 -50.4 5.2
-35.27 -0.02 -30.9 12.8
-34.92 -0.15 -35.3 7.5
-34.74 -0.08 -30.9 9.2
-33.74 -0.18 -74.3 9.8
-32.90 -0.15 -71.5 9.6
-32.25 -0.17 -71.5 9.2
-29.95 -0.00 -92.7 8.7
-26.61 -0.11 -114.2 7.3
-25.45 -0.14 -67.8 7.1
-24.86 0.01 -111.2 17.8
-21.21 -0.02 -144.4 10.5
-14.47 -0.15 -179.0 10.0
-13.73 -0.28 -191.4 7.5
-13.02 -0.21 -162.9 9.6
-12.68 -0.26 -217.2 10.7
-12.55 -0.23 -172.2 6.6""")

---

# APOGEE

In [ ]:
xmatch = Table.read('/Users/adrian/data/APOGEE_DR15beta/gaiadr2_allStar-t9-l31c-58158.fits')
xmatch.rename_column('apstar_id', 'APSTAR_ID')

In [ ]:
apogee = Table.read('/Users/adrian/data/APOGEE_DR15beta/allStar-t9-l31c-58158.fits')

In [ ]:
apogee_gaia = join(apogee, xmatch, keys='APSTAR_ID')
ndim_cols = [x for x in apogee_gaia.columns if apogee_gaia[x].ndim == 1]
apogee_gaia = apogee_gaia[ndim_cols]

# select distance stuff
apogee_gaia = apogee_gaia[(apogee_gaia['parallax'] < 1.) | ((apogee_gaia['parallax'] - apogee_gaia['parallax_error']) < 1.)]

g = GaiaData(apogee_gaia)
len(g)

In [ ]:
c = g.get_skycoord(distance=8*u.kpc, 
                   radial_velocity=np.array(g.VHELIO_AVG)*u.km/u.s)
c = gc.reflex_correct(c.transform_to(gc.GD1))

phi1 = c.phi1.wrap_at(180*u.deg)
phi2 = c.phi2

pm1 = c.pm_phi1_cosphi2.to(u.mas/u.yr)
pm2 = c.pm_phi2.to(u.mas/u.yr)

In [ ]:
gd1_mask = ((phi1 > -120*u.deg) & (phi1 < 40*u.deg) &
            (phi2 > -10*u.deg) & (phi2 < 5*u.deg))

phi_mask = mpl.patches.Path(phi_poly).contains_points(np.vstack((phi1, phi2)).T)
pm_mask = mpl.patches.Path(pm_poly).contains_points(np.vstack((pm1, pm2)).T)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 3))

ax.plot(phi1[gd1_mask], phi2[gd1_mask], 
        marker='o', mec='tab:blue', mew=1, mfc='none',
        ls='none', alpha=0.5)

ax.plot(gd1_phi1, gd1_phi2, 
        marker='.', ls='none', alpha=0.5, color='k')

ax.set_xlim(-100, 20)
ax.set_ylim(-10, 5)
ax.set_aspect('equal')

ax.set_xlabel(r'$\phi_1$')
ax.set_ylabel(r'$\phi_2$')

fig.tight_layout()
# fig.savefig('../plots/full-phi1phi2.png', dpi=250)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

ax.plot(pm1[gd1_mask & phi_mask], pm2[gd1_mask & phi_mask],  
        marker='.', ls='none', alpha=0.2, color='k')

pm_path = mpl.patches.Polygon(pm_poly, 
                              facecolor='none', edgecolor='k')
ax.add_patch(pm_path)

ax.scatter(model_c_ref.pm_phi1_cosphi2.to(u.mas/u.yr), 
           model_c_ref.pm_phi2.to(u.mas/u.yr), 
           zorder=-100, color='#aaaaaa')

ax.set_xlim(-15, 15)
ax.set_ylim(-15, 15)

ax.set_xlabel(r'$\mu_1 \, \cos\phi_2$')
ax.set_ylabel(r'$\mu_2$')
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

rv = g.VHELIO_AVG

ax.scatter(phi1[gd1_mask & phi_mask & pm_mask], rv[gd1_mask & phi_mask & pm_mask], 
           marker='o', color='tab:blue', linewidth=1, facecolor='none',
           alpha=1, s=25, label='APOGEE')

drv = np.abs(phi1_to_rv(phi1.value) - rv)
rv_members_mask = drv < 30.

ax.scatter(phi1[gd1_mask & phi_mask & pm_mask & rv_members_mask],
           rv[gd1_mask & phi_mask & pm_mask & rv_members_mask], 
           color='tab:red', label='[Fe/H]+Gaia members')

ax.plot(kop_vr['phi1'], kop_vr['vr'], 
        marker='o', ls='none', alpha=0.75, color='k', 
        label='Koposov/SEGUE')

ax.scatter(model_c.phi1.wrap_at(180*u.deg),
           model_c.radial_velocity.to(u.km/u.s), 
           zorder=-100, color='#aaaaaa',
           label='model')

ax.set_xlim(-100, 20)
ax.set_ylim(-350, 350)

ax.set_xlabel(r'$\phi_1$ [deg]')

ax.legend(loc='upper right', fontsize=18)
ax.set_ylabel(r'$v_r$ [{0:latex_inline}]'.format(u.km/u.s))
fig.tight_layout()

In [ ]:
(gd1_mask & phi_mask & pm_mask & rv_members_mask).sum()

In [ ]:
feh_mask = (g.M_H < -1) & (g.M_H > -2.1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

ax.errorbar(g.M_H[gd1_mask & phi_mask & pm_mask], 
            g.ALPHA_M[(gd1_mask & phi_mask & pm_mask)], 
            xerr=g.M_H_ERR[(gd1_mask & phi_mask & pm_mask)], 
            yerr=g.ALPHA_M_ERR[(gd1_mask & phi_mask & pm_mask)], 
            marker='o', ls='none', color='k', ecolor='#aaaaaa', alpha=0.5)

ax.scatter(g.M_H[gd1_mask & phi_mask & pm_mask & rv_members_mask], 
           g.ALPHA_M[gd1_mask & phi_mask & pm_mask & rv_members_mask], 
           marker='o', color='tab:red', alpha=0.75, zorder=100, s=50)

ax.axvspan(-2, -1, lw=0, zorder=-100, color='tab:blue', alpha=0.1)

ax.set_xlim(-2.5, 0.5)
ax.set_ylim(-0.2, 0.7)

In [ ]:
(gd1_mask & phi_mask & pm_mask & rv_members_mask & feh_mask).sum()

In [ ]:
members = g[gd1_mask & phi_mask & pm_mask & rv_members_mask & feh_mask]

In [ ]:
members.J - members.K

In [ ]:
members.H

In [ ]:
members.LOGG

In [ ]:
members.TEFF

In [ ]:
members.get_skycoord(distance=False).transform_to(gc.GD1).phi1.wrap_at(180*u.deg)[0]

In [ ]:
members.VHELIO_AVG

# Check LAMOST

In [ ]:
lamost_g = GaiaData('/Users/adrian/data/LAMOST/lamost-dr3-gaia-dr2.fits')

In [ ]:
c = lamost_g.get_skycoord(distance=8*u.kpc,
                          radial_velocity=lamost_g.rv)

c = gc.reflex_correct(c.transform_to(gc.GD1))

phi1 = c.phi1.wrap_at(180*u.deg)
phi2 = c.phi2

pm1 = c.pm_phi1_cosphi2.to(u.mas/u.yr)
pm2 = c.pm_phi2.to(u.mas/u.yr)

rv = c.radial_velocity
rv_err = lamost['rv_err']

In [ ]:
gd1_mask = ((phi1 > -120*u.deg) & (phi1 < 40*u.deg) &
            (phi2 > -10*u.deg) & (phi2 < 5*u.deg))

phi_mask = mpl.patches.Path(phi_poly).contains_points(np.vstack((phi1, phi2)).T)
pm_mask = mpl.patches.Path(pm_poly).contains_points(np.vstack((pm1, pm2)).T)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 3))

ax.plot(phi1, phi2, 
        marker='o', mec='tab:blue', mew=1, mfc='none',
        ls='none', alpha=0.5)

ax.plot(gd1_phi1, gd1_phi2, 
        marker='.', ls='none', alpha=0.5, color='k')

ax.set_xlim(-100, 20)
ax.set_ylim(-10, 5)
ax.set_aspect('equal')

ax.set_xlabel(r'$\phi_1$')
ax.set_ylabel(r'$\phi_2$')

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

ax.scatter(phi1[gd1_mask & phi_mask & pm_mask], rv[gd1_mask & phi_mask & pm_mask], 
           marker='o', color='tab:blue', linewidth=1, facecolor='none',
           alpha=1, s=25, label='APOGEE')

drv = np.abs(phi1_to_rv(phi1.value) - rv.value)
rv_members_mask = drv < 30.

ax.errorbar(phi1[gd1_mask & phi_mask & pm_mask & rv_members_mask].value,
            rv[gd1_mask & phi_mask & pm_mask & rv_members_mask].value, 
            yerr=np.array(rv_err[gd1_mask & phi_mask & pm_mask & rv_members_mask]),
            color='tab:red', label='rv cands', ls='none', marker='.')

ax.plot(kop_vr['phi1'], kop_vr['vr'], 
        marker='o', ls='none', alpha=0.75, color='k', 
        label='Koposov/SEGUE')

ax.scatter(model_c.phi1.wrap_at(180*u.deg),
           model_c.radial_velocity.to(u.km/u.s), 
           zorder=-100, color='#aaaaaa',
           label='model')

ax.set_xlim(-100, 20)
ax.set_ylim(-350, 350)

ax.set_xlabel(r'$\phi_1$ [deg]')

# ax.legend(loc='upper right', fontsize=18)
ax.set_ylabel(r'$v_r$ [{0:latex_inline}]'.format(u.km/u.s))
fig.tight_layout()

In [ ]:
feh_mask = lamost['feh'] < -1.4

In [ ]:
plt.hist(lamost['feh'][gd1_mask & phi_mask & pm_mask & rv_members_mask], 
         bins=np.linspace(-2.5, 0.5, 32));

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 3))

ax.plot(phi1[gd1_mask & phi_mask & pm_mask & rv_members_mask & feh_mask], 
        phi2[gd1_mask & phi_mask & pm_mask & rv_members_mask & feh_mask], 
        marker='o', color='tab:red', ls='none', alpha=1)

ax.plot(gd1_phi1, gd1_phi2, 
        marker='.', ls='none', alpha=0.5, color='k')

# ax.set_xlim(-100, 20)
# ax.set_ylim(-10, 5)

ax.set_xlim(-60, -20)
ax.set_ylim(-5, 5)

ax.set_aspect('equal')

ax.set_xlabel(r'$\phi_1$')
ax.set_ylabel(r'$\phi_2$')

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

ax.errorbar(phi1[gd1_mask & phi_mask & pm_mask & rv_members_mask & feh_mask].value,
            rv[gd1_mask & phi_mask & pm_mask & rv_members_mask & feh_mask].value, 
            yerr=np.array(rv_err[gd1_mask & phi_mask & pm_mask & rv_members_mask & feh_mask]),
            color='tab:red', label='rv cands', ls='none', marker='.')

ax.plot(kop_vr['phi1'], kop_vr['vr'], 
        marker='o', ls='none', alpha=0.75, color='k')

ax.set_xlim(-60, -20)
ax.set_ylim(-200, 100)

ax.set_xlabel(r'$\phi_1$')
# ax.set_ylabel(r'$\phi_2$')

fig.tight_layout()